# FFT and Timeseries tutorial

In [ ]:
import datetime
import numpy as np
import scipy as sp
import scipy.fftpack
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Utilisation des fonctions de type FFT sur des données météorologique

Télécharger le fichier weather.csv à l'adresse :
- https://github.com/ipython-books/cookbook-2nd-data/weather.csv

### Objectifs

- Analyse des données de températures sur plusieurs années
- Calcul de la FFT sur ces données
- Calcul de la PSD
- affichage des tendances globales de températures

In [ ]:
import os
data_dir = '/work/irlin355_1/gratienj/BigData/GitHub/DataAnalysisFramework/data'
filename = os.path.join(data_dir,'weather.csv')
df0 = pd.read_csv(filename,
                  na_values=(-9999),
                  parse_dates=['DATE'])

df = df0[df0['DATE'] >= '19940101']

df.head()


#### Calcul température moyenne

In [ ]:
df_avg = df.dropna().groupby('DATE').mean()

df_avg.head()


In [ ]:
date = pd.to_datetime(df_avg.index)
temp = (df_avg['TMAX'] + df_avg['TMIN']) / 20.
N = len(temp)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
temp.plot(ax=ax, lw=.5)
ax.set_ylim(-10, 40)
ax.set_xlabel('Date')
ax.set_ylabel('Mean temperature')

#### Cacul de la FFT

In [ ]:
temp_fft = sp.fftpack.fft(temp.values)

#### Calcul de la PSD

In [ ]:
temp_psd = np.abs(temp_fft) ** 2

In [ ]:
fftfreq = sp.fftpack.fftfreq(len(temp_psd), 1. / 365)

In [ ]:
i = fftfreq > 0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(fftfreq[i], 10 * np.log10(temp_psd[i]))
ax.set_xlim(0, 5)
ax.set_xlabel('Frequency (1/year)')
ax.set_ylabel('PSD (dB)')

#### Extraction des fréquences les plus importantes (FFT tronquée)

In [ ]:
temp_fft_bis = temp_fft.copy()
temp_fft_bis[np.abs(fftfreq) > 1.1] = 0

#### Calcul  de la courbe de température inverse de la FFT tronquée

Affichage de la courbe de température moyenne

In [ ]:
temp_slow = np.real(sp.fftpack.ifft(temp_fft_bis))

fig, ax = plt.subplots(1, 1, figsize=(6, 3))
temp.plot(ax=ax, lw=.5)
ax.plot_date(date, temp_slow, '-')
ax.set_xlim(datetime.date(1994, 1, 1),
            datetime.date(2000, 1, 1))
ax.set_ylim(-10, 40)
ax.set_xlabel('Date')
ax.set_ylabel('Mean temperature')
